# Downloading Necessary Libraries

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 datasets==2.14.6


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not c

In [ ]:
import os
import torch
from datasets import load_dataset, DatasetDict, Dataset, concatenate_datasets
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoModelForQuestionAnswering,
    AutoModelForCausalLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Hugging Face Logins

In [ ]:
import os
from google.colab import userdata
os.environ["HF_Token"] = userdata.get('HF_Token')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# Downloading pre-trained Fine-tuned model(1 epoch) from Hugging Face Repository

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel


base_model_name = "NousResearch/Llama-2-7b-chat-hf"  # Base model
repo_id = "DiatWork/llama2-Chat-Mental-Health"  # Repository with LoRA adapter

# Load base model and LoRA adapter from Hugging Face
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto", offload_folder="offload")
model_id = PeftModel.from_pretrained(base_model, repo_id)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

# Defining Q-LoRa Parameters

In [ ]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 10

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

# Data Loading and Transformation

In [ ]:
#split the dataset
dataset = load_dataset("ShenLab/MentalChat16K")

train_test_split= dataset['train'].train_test_split(test_size=0.2)
train_dataset= train_test_split['train']
test_dataset= train_test_split['test']

train_valid_split= train_dataset.train_test_split(test_size=0.1)
train_dataset = train_valid_split["train"]
validation_dataset = train_valid_split["test"]

# Combine into a DatasetDict
split_dataset = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test": test_dataset
})

print(split_dataset)

dataset= split_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/download/streaming_download_manager.py:765: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)
/usr/local/lib/python3.10/dist-packages/datasets/download/streaming_download_manager.py:765: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 11580
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1287
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 3217
    })
})


In [ ]:
#function to transform prompt
def transform_example(example):

    instruction = example['instruction'].strip()
    input_text = example['input'].strip() if example['input'] else ""
    output_text = example['output'].strip()


    if input_text:
        formatted_text = f"<s>[INST] {instruction} {input_text} [/INST] {output_text} </s>" #for cases when ther's no INST
    else:
        formatted_text = f"<s>[INST] {instruction} [/INST] {output_text} </s>"

    return {'text': formatted_text}

# transformer function call
transformed_dataset = DatasetDict({
    split: data.map(transform_example, remove_columns=data.column_names)
    for split, data in dataset.items()
})

print(transformed_dataset['train'][0]['text'])

Map:   0%|          | 0/11580 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

Map:   0%|          | 0/3217 [00:00<?, ? examples/s]

<s>[INST] You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description. 
The assistant gives helpful, comprehensive, and appropriate answers to the user's questions. Lately, I've been feeling really down and unmotivated. It's like a cloud of sadness that hangs over me all the time. I struggle to find joy in things I used to love, and even simple tasks feel overwhelming. My goal today is to understand why I'm feeling this way and learn strategies to lift myself out of this depressive state. [/INST] It can be challenging when you're experiencing feelings of sadness and a lack of motivation. It's important to remember that everyone's experience with depression is unique, but there are some strategies that may help improve your mood and lift yourself out of this state.

One strategy you could try is engaging in activities that used to bring you joy, even if you don't feel like doing them at first. By participating in th

In [ ]:
transformed_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 11580
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1287
    })
    test: Dataset({
        features: ['text'],
        num_rows: 3217
    })
})

# Re-Training model on 10 epochs

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)



base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_dataset["train"],         # Specify only the train split
    eval_dataset=transformed_dataset["validation"],     # Specify validation split
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)



Your GPU supports bfloat16: accelerate training with bf16=True


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

# Saving Model in Hugging Face Repository

In [ ]:
# Train model
trainer.train()


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,1.486200
50,1.757900
75,1.393300
100,1.492700
125,1.235800
150,1.190600
175,1.062500
200,0.982700
225,0.969800
250,0.890600


OSError: Can't load the configuration of 'PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_

In [ ]:
os.environ["WANDB_MODE"] = "disabled"

new_model = "llama2-MentalHealth-Finetune-10epoch"

In [ ]:
from pathlib import Path
import torch

# Save model from dir. to repo.
repo_id = "DiatWork/llama2-MentalHealth-Finetune-10epoch"
directory = "./4bit_model"


Path(directory).mkdir(parents=True, exist_ok=True)
tokenizer.save_pretrained(directory)
model.config.save_pretrained(directory)

torch.save(model.state_dict(), f"{directory}/pytorch_model.bin")


In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
from huggingface_hub import upload_folder
repo_id= "DiatWork/llama2-MentalHealth-Finetune-10epoch"

# Upload trained model to repository
upload_folder(folder_path=directory, repo_id=repo_id)


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/4.42G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DiatWork/llama2-MentalHealth-Finetune-10epoch/commit/dc879c823060b21848cc540b6df69f27ae703f77', commit_message='Upload folder using huggingface_hub', commit_description='', oid='dc879c823060b21848cc540b6df69f27ae703f77', pr_url=None, repo_url=RepoUrl('https://huggingface.co/DiatWork/llama2-MentalHealth-Finetune-10epoch', endpoint='https://huggingface.co', repo_type='model', repo_id='DiatWork/llama2-MentalHealth-Finetune-10epoch'), pr_revision=None, pr_num=None)

In [ ]:
adapters= "./llama2-MentalHealth-Finetune-10epoch"

upload_folder(folder_path= adapters, repo_id=repo_id)

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DiatWork/llama2-MentalHealth-Finetune-10epoch/commit/5951a4a9c0aa115a34a090186abb009bdb822161', commit_message='Upload folder using huggingface_hub', commit_description='', oid='5951a4a9c0aa115a34a090186abb009bdb822161', pr_url=None, repo_url=RepoUrl('https://huggingface.co/DiatWork/llama2-MentalHealth-Finetune-10epoch', endpoint='https://huggingface.co', repo_type='model', repo_id='DiatWork/llama2-MentalHealth-Finetune-10epoch'), pr_revision=None, pr_num=None)

# Load Model from Hugging Face

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel


base_model_name = "NousResearch/Llama-2-7b-chat-hf"  # Base model
repo_id = "DiatWork/llama2-MentalHealth-Finetune-10epoch"  # Repository link

# Load base model and LoRA adapter from Hugging Face
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto", offload_folder="offload")
model = PeftModel.from_pretrained(base_model, repo_id)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

# Inference

In [ ]:
import torch
import textwrap

# Input with Prompt
input_text = "<s>[INST] You are a helpful mental health assistant. I'm very stressed because I have a Job Interview, though I have practiced for it but this is my first interview so I'm anxious. How to deal with this stress? [/INST] "

inputs = tokenizer(input_text, return_tensors="pt").to(model.device). # Tokenize the input

# Generate response
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=400)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True).strip() # Tokenize the output

# Presentation
if "[INST]" in decoded_output and "[/INST]" in decoded_output:
    question, model_answer = decoded_output.split("[/INST]", 1)
else:
    question, model_answer = decoded_output, ""  # Handle cases where split fails


question = "\n".join(textwrap.wrap(question, width=80))
model_answer = "\n".join(textwrap.wrap(model_answer.strip(), width=80))

print("Question:\n", question)
print("\nModel Answer:\n", model_answer)
print("\n" + "=" * 80 + "\n")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=Tr

Question:
 [INST] You are a helpful mental health assistant. I'm very stressed because I
have a Job Interview, though I have practiced for it but this is my first
interview so I'm anxious. How to deal with this stress?

Model Answer:
 Acknowledging your stress and recognizing that it's a normal part of the job
interview process is an important first step in managing it. Here are some
suggestions to help you deal with your stress:  1. Take deep breaths: Before
your interview, take a few moments to focus on your breath. Inhale deeply
through your nose, hold for a few seconds, and then exhale slowly through your
mouth. This simple technique can help calm your nervous system and reduce
anxiety.  2. Visualize success: Imagine yourself acing the interview and landing
your dream job. Visualizing positive outcomes can boost your confidence and
alleviate stress. Picture yourself answering questions confidently and making a
strong impression on the interviewer.  3. Practice positive self-talk: R

In [ ]:
# Input with Prompt
input_text = "<s>[INST] You are a helpful mental health assistant. I'm very stressed because I have a group project in last semester of my college and I want to get first honours, I'm scared. Please help me cope with stress. [/INST]"

inputs = tokenizer(input_text, return_tensors="pt").to(model.device). # Tokenize the input

# Generate response
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=200)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True).strip() # Tokenize the output

# Presentation
if "[INST]" in decoded_output and "[/INST]" in decoded_output:
    question, model_answer = decoded_output.split("[/INST]", 1)
else:
    question, model_answer = decoded_output, ""  # Handle cases where split fails


question = "\n".join(textwrap.wrap(question, width=80))
model_answer = "\n".join(textwrap.wrap(model_answer.strip(), width=80))

print("Question:\n", question)
print("\nModel Answer:\n", model_answer)
print("\n" + "=" * 80 + "\n")



Question:
 [INST] You are a helpful mental health assistant. I'm very stressed because I
have a group project in last semester of my college and I want to get first
honours, I'm scared. Please help me cope with stress.

Model Answer:
 It's completely normal to feel stressed when facing a challenging situation like
a group project, especially with the goal of obtaining first honors. Here are
some suggestions to help you cope with stress:  1. Break it down: Start by
breaking down the project into smaller, manageable tasks. This will make it feel
less overwhelming and more achievable. Create a to-do list or use a planner to
keep track of your progress.  2. Communication is key: Make sure to communicate
openly and effectively with your group members. Discuss roles, responsibilities,
and deadlines to ensure everyone is on the same page. Regularly check-in with
each other to share updates and provide support.  3. Time management: Allocate
specific time slots for working on the project. Prior

# Inference for Human Evaluation


In [ ]:
# Input with Prompt
input_text = "<s>[INST] You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description.  The assistant gives helpful, comprehensive, and appropriate answers to the user's questions. Another trigger for my anxiety is crowded places. Whenever I'm in a crowded environment, such as shopping malls or public transportation, I feel overwhelmed and suffocated. The noise, the people, and the lack of personal space make me extremely anxious. Just last week, I had to leave a store because it was too crowded, and I felt trapped and panicked. [/INST]"

inputs = tokenizer(input_text, return_tensors="pt").to(model.device). # Tokenize the input

# Generate response
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=300, top_p=0.9)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True).strip() # Tokenize the output

# Presentation
if "[INST]" in decoded_output and "[/INST]" in decoded_output:
    question, model_answer = decoded_output.split("[/INST]", 1)
else:
    question, model_answer = decoded_output, ""  # Handle cases where split fails


question = "\n".join(textwrap.wrap(question, width=80))
model_answer = "\n".join(textwrap.wrap(model_answer.strip(), width=80))

print("Question:\n", question)
print("\nModel Answer:\n", model_answer)
print("\n" + "=" * 80 + "\n")



/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Question:
 [INST] You are a helpful mental health counselling assistant, please answer the
mental health questions based on the patient's description.  The assistant gives
helpful, comprehensive, and appropriate answers to the user's questions. Another
trigger for my anxiety is crowded places. Whenever I'm in a crowded environment,
such as shopping malls or public transportation, I feel overwhelmed and
suffocated. The noise, the people, and the lack of personal space make me
extremely anxious. Just last week, I had to leave a store because it was too
crowded, and I felt trapped and panicked.

Model Answer:
 It can be challenging to navigate crowded places when you're experiencing
anxiety. One approach that may help is gradually exposing yourself to these
environments in a controlled way. You could start by visiting less crowded
places during off-peak hours or during the week, when the crowds may be smaller.
As you become more comfortable, you can gradually work your way up to busier
lo

In [ ]:
# Input with Prompt
input_text = "<s>[INST] You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description.  The assistant gives helpful, comprehensive, and appropriate answers to the user's questions.  I've been wrestling with the decision to visit my mother-in-law, torn between concern for her health and the disagreement with my husband about her safety. Memories of her last days weigh heavily on my mind, and I can't ignore the deterioration in her condition at 89. The stroke she suffered left her with limited strength in her left leg, and her sedentary lifestyle only worsens her health. I've been considering the best approach to approach this delicate situation. [/INST]"

inputs = tokenizer(input_text, return_tensors="pt").to(model.device). # Tokenize the input

# Generate response
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=300, top_p=0.9)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True).strip() # Tokenize the output

# Presentation
if "[INST]" in decoded_output and "[/INST]" in decoded_output:
    question, model_answer = decoded_output.split("[/INST]", 1)
else:
    question, model_answer = decoded_output, ""  # Handle cases where split fails


question = "\n".join(textwrap.wrap(question, width=80))
model_answer = "\n".join(textwrap.wrap(model_answer.strip(), width=80))

print("Question:\n", question)
print("\nModel Answer:\n", model_answer)
print("\n" + "=" * 80 + "\n")



Question:
 [INST] You are a helpful mental health counselling assistant, please answer the
mental health questions based on the patient's description.  The assistant gives
helpful, comprehensive, and appropriate answers to the user's questions.  I've
been wrestling with the decision to visit my mother-in-law, torn between concern
for her health and the disagreement with my husband about her safety. Memories
of her last days weigh heavily on my mind, and I can't ignore the deterioration
in her condition at 89. The stroke she suffered left her with limited strength
in her left leg, and her sedentary lifestyle only worsens her health. I've been
considering the best approach to approach this delicate situation.

Model Answer:
 Your concern for your mother-in-law's wellbeing is a testament to your love and
care for her. It's understandable that you're torn between your desire to
support her and the disagreement with your husband about her safety. The
memories of her last days and the curren

In [ ]:
!pip install evaluate bert_score transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.0 MB/s eta 0:00:00


In [ ]:
dataset['test']

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 3217
})

# Testing on 20% Test Data

In [ ]:
import textwrap
import torch
import evaluate

# 20% test data for evaluation
test_data = dataset["test"]
subset_size= int(0.2 * len(test_data))

prompts = []
references = []

# Build prompts and references
for i in range(subset_size):
    sample = test_data[i]
    instruction = sample['instruction']
    input_text = sample['input']
    output_text = sample['output']

    if input_text:
        prompt = f"<s>[INST] {instruction} {input_text} [/INST]"
    else:
        prompt = f"<s>[INST] {instruction} [/INST]"

    prompts.append(prompt)
    references.append(output_text)  #reference answer

# Generate responses for each prompt and format them
generated_texts = []
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=True,
            top_p=0.9,
            top_k=50,
            temperature=0.7,
            use_cache=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the response
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    # Presentation
    try:
        question, model_answer = decoded_output.split("[/INST]", 1)
    except ValueError:
        question, model_answer = decoded_output, ""


    question = "\n".join(textwrap.wrap(question, width=80))
    model_answer = "\n".join(textwrap.wrap(model_answer, width=80))

    print("Question:\n", question)
    print("\nModel Answer:\n", model_answer)
    print("\n" + "="*80 + "\n")

    # Append decoded output to generated_texts for evaluation
    generated_texts.append(decoded_output)





/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Streaming output truncated to the last 5000 lines.
increased heart rate, and surges of adrenaline, finding relaxation techniques
that work for you can be beneficial. Deep breathing exercises are a


Question:
 [INST] You are a helpful mental health counselling assistant, please answer the
mental health questions based on the patient's description.  The assistant gives
helpful, comprehensive, and appropriate answers to the user's questions.
Recently, I've been feeling an overwhelming amount of stress which is
interfering with my ability to function properly. My goal for this counseling
session is to find ways to reduce my stress levels and regain a sense of balance
in my life.  The primary trigger for my stress is related to academic pressures.
I am currently a college student, and the workload has become increasingly
demanding. The expectation to excel academically while balancing extracurricular
activities and maintaining social connections often feels burdensome.  Specific
examples o

KeyboardInterrupt: 

In [16]:
!pip install rouge_score

#Initialize evaluation metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

bleu_score = bleu.compute(predictions=generated_texts, references=[[ref] for ref in references])
rouge_score = rouge.compute(predictions=generated_texts, references=references)
bert_score = bertscore.compute(predictions=generated_texts, references=references, lang="en")

# Print Evaluation Results
print("BLEU Score:", bleu_score)
print("ROUGE Score:", rouge_score)
print("BERTScore (Precision, Recall, F1):", bert_score["precision"], bert_score["recall"], bert_score["f1"])

from huggingface_hub import Repository

repo = Repository(local_dir="./evaluation_results", clone_from="DiatWork/llama2-MentalHealth-Finetune-10epoch")

# Save metrics to JSON file
evaluation_metrics = {
    "ROUGE": rouge_score,
    "BERTScore": bert_score,
    "BLEU": bleu_score,
}

import json

output_file = "evaluation_metrics.json"
with open(output_file, "w") as json_file:
    json.dump(evaluation_metrics, json_file, indent=4)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

...

Download file tokenizer.model:   5%|4         | 24.0k/488k [00:00<?, ?B/s]

Download file adapter_model.bin:   0%|          | 8.00k/128M [00:00<?, ?B/s]

Clean file tokenizer.model:   0%|          | 1.00k/488k [00:00<?, ?B/s]

Clean file adapter_model.bin:   0%|          | 1.00k/128M [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/4.12G [00:00<?, ?B/s]

In [17]:
import json
import numpy as np

file_path = '/content/evaluation_metrics.json'

with open(file_path, 'r') as file:
    gemma_data = json.load(file)

# Calculate Avg. BERTScore
bert_score = gemma_data["BERTScore"]

precision_avg = np.mean(bert_score["precision"])
recall_avg = np.mean(bert_score["recall"])
f1_avg = np.mean(bert_score["f1"])

precision_avg, recall_avg, f1_avg


(0.8640052236399605, 0.8754148573551364, 0.869581125083479)